In [6]:
import os, time, re, random, glob
import jieba
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

/Users/zeyesun/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
NOTE: Redirects are currently not supported in Windows or MacOs.


In [7]:
# device="cuda:0"
device="cpu"

In [1]:
import sentencepiece
model_file = "/Users/zeyesun/Documents/Data/models/pangu-350M/vocab.model"
sp = sentencepiece.SentencePieceProcessor()
sp.Load(model_file=model_file)

True

In [14]:
l = [23]
l = np.array(l)
print(type(l))
sp.decode(l.tolist())

<class 'numpy.ndarray'>


'有'

# SFT

In [ ]:
model_name_or_path = "D:\\Data\\models\\pangu_2_6B"
# model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, use_cache=False)
model.to(device)
# print(model.device)

In [ ]:
# st = torch.load(os.path.join(model_name_or_path, "pytorch_model.bin"), map_location="cpu")
# print(len(st))
# # st.keys()

In [ ]:
text_generator = TextGenerationPipeline(model, tokenizer, device=device)
# greedy search
print(text_generator("中国和美国和日本和法国和加拿大和澳大利亚的首都分别是哪里？", max_length=50))